# **<p style="text-align:center;">  <span style="color:white;">Flask Project Made by Ahmed Basiony</span></p>**


 ## <span style="color:orange">*Order Management project using Flask and SQL Server, with optional HTML integration*</span>

# <span style="color:red">Create a database called InventoryDB. </span>
1- Customers (Pre-loaded from API) .

CREATE TABLE Customers (

customer_id INT PRIMARY KEY,

full_name NVARCHAR(100),

email NVARCHAR(100),

city NVARCHAR(50));

In [63]:
import pandas as pd
import pyodbc
from flask import Flask, render_template, redirect , request, url_for
from pathlib import Path
import requests


In [64]:

### crate database called InventoryDB
conn = pyodbc.connect('Driver={SQL Server};'
                    'Server=.\\SQLEXPRESS;'
                    'Database=mydata;'
                    'Trusted_Connection=yes;')


cursor = conn.cursor()
cursor.execute("CREATE DATABASE InventoryDB;")
cursor.commit()
cursor.close()
conn.close()

In [65]:

### connect to database called InventoryDB
conn = pyodbc.connect('Driver={SQL Server};'
                    'Server=.\\SQLEXPRESS;'
                    'Database=InventoryDB;'
                    'Trusted_Connection=yes;')
cursor = conn.cursor()
### create table called Customers
cursor.execute("""CREATE TABLE Customers 
                (customer_id INT PRIMARY KEY, 
                full_name NVARCHAR(100),
                email NVARCHAR(100),
                city NVARCHAR(50));"""
                )
cursor.commit()



# <span style="color:red">Create a database called InventoryDB. </span>
2- Products (Pre-loaded from API)

CREATE TABLE Products (

product_id INT PRIMARY KEY,

name NVARCHAR(100),

price DECIMAL(10,2),

stock_quantity INT)


In [66]:

### create table called Products
cursor.execute("""CREATe TABLE Products
                ( product_id INT PRIMARY KEY,
                product_name NVARCHAR(100),
                price DECIMAL(10, 2), 
                quantity INT);"""
                )

# <span style="color:red">Create a database called InventoryDB. </span>
3- Orders Table

CREATE TABLE Orders (

order_id INT PRIMARY KEY IDENTITY(1,1),

customer_id INT NOT NULL,

product_id INT NOT NULL,

quantity INT NOT NULL,

price DECIMAL(10,2) NOT NULL,

order_date DATETIME DEFAULT GETDATE(),

status NVARCHAR(20) DEFAULT 'Pending',

FOREIGN KEY (customer_id) REFERENCES Customers(customer_id),

FOREIGN KEY (product_id) REFERENCES Products(product_id));



In [67]:

### create table called Orders
cursor.execute("""
                CREATE TABLE Orders
                (order_id INT PRIMARY KEY IDENTITY(1,1), 
                customer_id INT, 
                product_id INT, 
                order_date DATETIME DEFAULT GETDATE(),
                Status NVARCHAR(20) DEFAULT 'Pending', 
                FOREIGN KEY (customer_id) REFERENCES Customers(customer_id), 
                FOREIGN KEY (product_id) REFERENCES Products(product_id));
                """)
cursor.commit()
cursor.close()
conn.close()

 ### <span style="color:orange">*After creating the database and tables using Python to connect to SQL Server and your database, use the Requests library to download the data of employees and products from the following APIs and insert them into the tables*</span>

In [68]:

### getting data of customers and products
response = requests.get("https://raw.githubusercontent.com/MohammedHameds/Flask-project/refs/heads/main/dataset/customers.json")
customers = pd.DataFrame(response.json())
response = requests.get("https://raw.githubusercontent.com/MohammedHameds/Flask-project/refs/heads/main/dataset/products.json")
products = pd.DataFrame(response.json())

# <span style="color:red">Insert the data into the database. </span>


In [69]:
con = pyodbc.connect('Driver={SQL Server};'
                    'Server=.\\SQLEXPRESS;'
                    'Database=InventoryDB;'
                    'Trusted_Connection=yes;')
cursor = con.cursor()
cursor.fast_executemany = True

cursor.executemany(
    "INSERT INTO Customers VALUES (?, ?, ?, ?)",
    customers.values.tolist()
)
con.commit()

cursor.executemany(
    "INSERT INTO Products VALUES (?, ?, ?, ?)",
    products.values.tolist()
)
con.commit()

cursor.close()  

 ### <span style="color:orange">*Now that the database is ready, let's create a Flask app that includes some methods.*</span>


# <span style="color:red">1- Create Order Method: POST - Endpoint: “/orders” </span>
# <span style="color:red">2- Dsiplay Orders Method: GET Endpoint: /allorders</span>
# <span style="color:red">3- Get Order Details Method: GET Endpoint: /orders/<order_id> </span>




In [ ]:
from flask import Flask, render_template, request, redirect, url_for, session
from pathlib import Path
import pyodbc

# -----------------------------
# Paths
# -----------------------------
BASE_DIR = Path.cwd().parent
TEMPLATES_DIR = BASE_DIR / "templates"
STATIC_DIR = BASE_DIR / "static"

app = Flask(__name__, template_folder=str(TEMPLATES_DIR), static_folder=str(STATIC_DIR))
app.secret_key = "any_secret_key"


# -----------------------------
# Database Connection
# -----------------------------
def get_conn():
    return pyodbc.connect(
        "Driver={SQL Server};"
        "Server=.\\SQLEXPRESS;"
        "Database=InventoryDB;"
        "Trusted_Connection=yes;"
    )


# -----------------------------
# Routes
# -----------------------------
@app.route("/")
def home():
    return render_template("index.html")


@app.route("/login", methods=["GET", "POST"])
def login():
    if request.method == "POST":
        email = request.form.get("email", "").strip()

        conn = get_conn()
        cur = conn.cursor()
        cur.execute(
            "SELECT customer_id, full_name FROM Customers WHERE email = ?", (email,)
        )
        user = cur.fetchone()
        conn.close()

        if user:
            session["customer_id"] = int(user[0])
            session["customer_name"] = user[1]
            return redirect(url_for("home"))

        return render_template("login.html", error="Email not found")

    # GET
    return render_template("login.html")


@app.route("/logout")
def logout():
    session.clear()
    return redirect(url_for("home"))


@app.route("/add-order", methods=["POST"])
def add_order():
    customer_id = session.get("customer_id")
    if not customer_id:
        return redirect(url_for("login"))

    product_id = int(request.form["product_id"])

    conn = get_conn()
    cur = conn.cursor()

    try:
        
        cur.execute("""
            UPDATE Products
            SET quantity = quantity - 1
            WHERE product_id = ? AND quantity > 0
        """,(product_id,))

        
        if cur.rowcount == 0:
            conn.rollback()
            return "Out of stock", 400


        cur.execute("""
            INSERT INTO Orders (customer_id, product_id, order_date)
            VALUES (?, ?, GETDATE())
        """, (customer_id, product_id))

        conn.commit()
        return redirect(url_for("display_orders"))

    except Exception as e:
        conn.rollback()
        raise e

    finally:
        conn.close()


@app.route("/all_orders")
def display_orders():
    conn = get_conn()
    cur = conn.cursor()

    cur.execute(
        """
                SELECT
                o.order_id,
                o.order_date,
                c.full_name,
                p.product_name,
                p.price
                FROM Orders o
                JOIN Customers c ON c.customer_id = o.customer_id
                JOIN Products p ON p.product_id = o.product_id
                ORDER BY o.order_id DESC
        """
    )

    rows = cur.fetchall()
    conn.close()

    orders = [
        {
            "order_id": r[0],
            "order_date": r[1],
            "customer_name": r[2],
            "product_name": r[3],
            "price": float(r[4]),
        }
        for r in rows
    ]

    return render_template("display_orders.html", orders=orders)


@app.route("/orders/<int:order_id>")
def get_order_details(order_id):
    return render_template("get_order_details.html", order_id=order_id)


# -----------------------------
# Run App
# -----------------------------
if __name__ == "__main__":
    app.run(port=5050, debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5050
Press CTRL+C to quit
127.0.0.1 - - [22/Feb/2026 05:25:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Feb/2026 05:25:03] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [22/Feb/2026 05:25:03] "GET /static/images/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Feb/2026 05:25:03] "GET /static/images/home%20page%20Photo.jpg HTTP/1.1" 304 -
127.0.0.1 - - [22/Feb/2026 05:25:03] "GET /static/images/T1.jpg HTTP/1.1" 304 -
127.0.0.1 - - [22/Feb/2026 05:25:03] "GET /static/images/iphone%2014.jpg HTTP/1.1" 304 -
127.0.0.1 - - [22/Feb/2026 05:25:03] "GET /static/images/online-shopping.png HTTP/1.1" 304 -
127.0.0.1 - - [22/Feb/2026 05:25:03] "GET /static/images/samsung-55.jpg HTTP/1.1" 304 -
127.0.0.1 - - [22/Feb/2026 05:25:03] "GET /static/images/Mouse.jpg HTTP/1.1" 304 -
127.0.0.1 - - [22/Feb/2026 05:25:03] "GET /static/images/Keyboard.jpg HTTP/1.1" 304 -
127.0.0.1 - - [22/Feb/2026 05:25:03] "GET /static/images/B.jpg HTTP/1.1" 304 -
127.0.0.1 - - [22/Feb/2026 05: